### Create Antecedents and Consequent
- Import membership functions from `fuzzy_memberships.py` (universes and sets).
- Define 5 antecedents: BMI, Age, HighBP, Smoker, PhysActivity.
- Define 1 consequent: Risk (use `defuzzify_method='centroid'`).
- Attach membership functions to each variable:
  - BMI: under, healthy, over, obese
  - Age: young, middle, old
  - HighBP: normal, high
  - Smoker: no, yes
  - PhysActivity: inactive, active
  - Risk: vlow, low, med, high, vhigh
- Sanity check: each variable has all labels and names match the imports.

In [8]:
import pandas as pd
import numpy as np
import skfuzzy.control as ctrl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# --- Import all universe variables and MFs from the external file ---
try:
    from fuzzy_memberships import (
        bmi_universe, age_universe, binary_universe, risk_universe,
        bmi_under, bmi_healthy, bmi_over, bmi_obese,
        age_young, age_middle, age_old,
        bp_normal, bp_high,
        smoke_no, smoke_yes,
        act_inactive, act_active,
        risk_vlow, risk_low, risk_med, risk_high, risk_vhigh
    )
    print("Fuzzy membership functions imported successfully from fuzzy_memberships.py.")
except ImportError:
    print("Error: Could not import from fuzzy_memberships.py. Ensure the file exists.")
    exit()

# --- Load Cleaned Data ---
try:
    df = pd.read_csv("../data/diabetes_clean.csv")
    print("Cleaned data loaded successfully.")
except FileNotFoundError:
    print("Error: 'diabetes_clean.csv' not found. Ensure the file path is correct.")
    exit()

Fuzzy membership functions imported successfully from fuzzy_memberships.py.
Cleaned data loaded successfully.


In [3]:
# D3 — Create Antecedents and Consequent
bmi = ctrl.Antecedent(bmi_universe, 'BMI')
age = ctrl.Antecedent(age_universe, 'Age')
highbp = ctrl.Antecedent(binary_universe, 'HighBP')
smoker = ctrl.Antecedent(binary_universe, 'Smoker')
physact = ctrl.Antecedent(binary_universe, 'PhysActivity')

risk = ctrl.Consequent(risk_universe, 'Risk', defuzzify_method='centroid')

bmi['under'] = bmi_under
bmi['healthy'] = bmi_healthy
bmi['over'] = bmi_over
bmi['obese'] = bmi_obese

age['young'] = age_young
age['middle'] = age_middle
age['old'] = age_old

highbp['normal'] = bp_normal
highbp['high'] = bp_high

smoker['no'] = smoke_no
smoker['yes'] = smoke_yes

physact['inactive'] = act_inactive
physact['active'] = act_active

risk['vlow'] = risk_vlow
risk['low'] = risk_low
risk['med'] = risk_med
risk['high'] = risk_high
risk['vhigh'] = risk_vhigh